In [1]:
#importing necessary libraires
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import datetime as dt
from sklearn.preprocessing import StandardScaler
import pandas_datareader.data as web
import math
import xlsxwriter
import os
import re

In [41]:
#loading data
df = pd.read_excel('Tweets/AAL.xlsx', header = 0, sheet_name = "Stream")
df.head()
stock = "AAL"

In [43]:
#removing links from the tweets
for idx, tweet in enumerate(df['Tweet content']):
    df['Tweet content'][idx] = re.sub(r'http\S+', '', tweet)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
#calculating sentiment scores
date = df['Date']
time = df['Hour']
tweet = df['Tweet content']
followers = df['Followers']
symbols = df['Symbols']



new_dict = {'date': date, 'time': time, 'tweet': tweet, 'follwers': followers, 'symbols': symbols}
new_df = pd.DataFrame(new_dict)
new_df

,date,time,tweet,follwers,symbols
0,2016-06-15,09:26,"Why American Airlines Group Inc $AAL, Novo Nor...",2039.0,"$AAL,$NVO,$SYF"
1,2016-06-15,08:18,Yesterday's top #FTSE100 fallers - Anglo Ameri...,1792.0,"$AAL,$ANTO,$SKY,$BKG,$BLT"
2,2016-06-15,08:06,SA_QuickIdeas: 5 Large-Cap Stocks With The Low...,1589.0,"$AAL,$DAL,$HPQ,$TTM,$UAL,$VLO"
3,2016-06-15,08:04,JPMorgan Chase &amp; Co. Reiterates Underweigh...,771.0,$AAL
4,2016-06-15,08:01,$DAL $AAL:\n\n5 Large-Cap Stocks With The Lowe...,788.0,"$DAL,$AAL"
...,...,...,...,...,...
6502,2016-03-10,13:02,#Cuba: U.S. #airlines vie 4 #latinamerica flig...,248.0,"$AAL,$DAL,$JBLU,$LUV,$UAL,$ALK,$VWO,$EEM,$FM,$ILF"
6503,2016-03-10,12:36,Our New Stock Pick Is On $PSNP &amp; It's Pick...,12543.0,"$PSNP,$DROP,$AAL,$SVMI"
6504,2016-03-10,12:34,Our Hot New Stock Alert Is $PSNP! Things Are R...,19713.0,"$PSNP,$DROP,$AAL,$SVMI"
6505,2016-03-10,12:32,This Mineral Exploration Company Is Heating Up...,12556.0,"$DROP,$AAL,$SVMI"


In [45]:
analyzer = SentimentIntensityAnalyzer()


neg = []
neu = []
pos = []
com = []

for tweet in new_df['tweet']:
    sentiment = analyzer.polarity_scores(tweet)
    neg.append(sentiment['neg'])
    neu.append(sentiment['neu'])
    pos.append(sentiment['pos'])
    com.append(sentiment['compound'])

    
new_df['negative'] = neg
new_df['neutral'] = neu
new_df['positive'] = pos
new_df['compound'] = com

In [46]:
new_df.head()

,date,time,tweet,follwers,symbols,negative,neutral,positive,compound
0,2016-06-15,09:26,"Why American Airlines Group Inc $AAL, Novo Nor...",2039.0,"$AAL,$NVO,$SYF",0.194,0.806,0.000,-0.6249
1,2016-06-15,08:18,Yesterday's top #FTSE100 fallers - Anglo Ameri...,1792.0,"$AAL,$ANTO,$SKY,$BKG,$BLT",0.000,0.899,0.101,0.2023
2,2016-06-15,08:06,SA_QuickIdeas: 5 Large-Cap Stocks With The Low...,1589.0,"$AAL,$DAL,$HPQ,$TTM,$UAL,$VLO",0.167,0.833,0.000,-0.3818
3,2016-06-15,08:04,JPMorgan Chase &amp; Co. Reiterates Underweigh...,771.0,$AAL,0.000,1.000,0.000,0.0000
4,2016-06-15,08:01,$DAL $AAL:\n\n5 Large-Cap Stocks With The Lowe...,788.0,"$DAL,$AAL",0.245,0.755,0.000,-0.3818


In [48]:
#filtering tweets for 70 days

df_tweets = new_df[(new_df['date'] >= '2016-04-01') & (new_df['date'] <= '2016-06-11')]

df_tweets

,date,time,tweet,follwers,symbols,negative,neutral,positive,compound
299,2016-06-11,23:16,"DallasBizNews: ICYMI: American Airlines, South...",1829.0,"$AAL,$LUV",0.0,1.000,0.000,0.0000
300,2016-06-11,23:06,"ICYMI: American Airlines, Southwest can start ...",27066.0,"$AAL,$LUV",0.0,1.000,0.000,0.0000
301,2016-06-11,22:35,RT @WallStJesus: JUMPS IN NASDAQ SHORTS: $AAPL...,17.0,"$AAPL,$AMD,$BIDU,$AAL,$QCOM,$ENDP,$CAR,$GILD,$...",0.0,1.000,0.000,0.0000
302,2016-06-11,22:10,RT @WallStJesus: JUMPS IN NASDAQ SHORTS: $AAPL...,339.0,"$AAPL,$AMD,$BIDU,$AAL,$QCOM,$ENDP,$CAR,$GILD,$...",0.0,1.000,0.000,0.0000
303,2016-06-11,21:44,TUP Prev Close\nSee Here ---&gt; \n\n$TUP $ORE...,78.0,"$TUP,$OREX,$AAL",0.0,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...
5733,2016-04-01,02:03,$AAL:\n\nAmerican Airlines Group’s (AAL) “Outp...,699.0,$AAL,0.0,1.000,0.000,0.0000
5734,2016-04-01,01:50,American Airlines Group's outperform rating re...,4519.0,$AAL,0.0,0.794,0.206,0.3818
5735,2016-04-01,01:47,RT SeekingAlpha: Airplane And Hospitality Indu...,8560.0,"$HOT,$MAR,$JBLU,$DAL,$LUV,$AAL,$ALK",0.0,1.000,0.000,0.0000
5736,2016-04-01,01:45,American Airlines Group's outperform rating re...,4001.0,$AAL,0.0,0.794,0.206,0.3818


In [29]:
start = dt.datetime(2016, 4, 2)
end =  dt.datetime(2016, 6, 11)

df_stock = web.DataReader(stock, 'yahoo', start, end)

In [49]:
df_stock

,High,Low,Open,Close,Volume_stock,Adj_Close_stock,HiLo_vola_stock,Pct_change_stock,Pct_change_stock_scaled
Date,,,,,,,,,
2016-04-04,40.240002,39.150002,39.810001,39.369999,7831200.0,37.726421,2.889222,-1.105256,-0.479860
2016-04-05,39.380001,38.740002,39.130001,38.930000,7648400.0,37.304790,1.715596,-0.511119,-0.117639
2016-04-06,39.070000,38.340000,39.070000,38.900002,5650000.0,37.276047,1.958361,-0.435112,-0.071301
2016-04-07,39.049999,37.910000,38.660000,38.360001,8206700.0,36.758587,3.101315,-0.775994,-0.279122
2016-04-08,38.720001,38.119999,38.700001,38.500000,6087400.0,36.892742,1.626342,-0.516798,-0.121102
2016-04-11,39.549999,38.669998,38.790001,38.730000,7932300.0,37.113148,2.371130,-0.154683,0.099665
2016-04-12,39.430000,38.759998,38.820000,39.189999,6426000.0,37.553940,1.784106,0.953114,0.775041
2016-04-13,40.009998,38.889999,39.299999,39.939999,12493500.0,38.272629,2.926371,1.628497,1.186794
2016-04-14,41.660000,40.250000,40.340000,41.169998,10133400.0,39.451279,3.574028,2.057506,1.448342


In [31]:
df_stock.columns = ['High','Low','Open','Close','Volume_stock','Adj_Close_stock']

df_stock['HiLo_vola_stock'] = (df_stock['High'] - df_stock['Low']) / df_stock['Adj_Close_stock'] * 100.0
df_stock['Pct_change_stock'] = (df_stock['Close'] - df_stock['Open']) / df_stock['Open'] * 100.0

stock_1 = df_stock[['Pct_change_stock']].values.astype(float)

scaler = StandardScaler().fit(stock_1)

scaled_data = scaler.transform(stock_1)

df_stock['Pct_change_stock_scaled'] = scaled_data

df_stock.head()

,High,Low,Open,Close,Volume_stock,Adj_Close_stock,HiLo_vola_stock,Pct_change_stock,Pct_change_stock_scaled
Date,,,,,,,,,
2016-04-04,40.240002,39.150002,39.810001,39.369999,7831200.0,37.726421,2.889222,-1.105256,-0.479860
2016-04-05,39.380001,38.740002,39.130001,38.930000,7648400.0,37.304790,1.715596,-0.511119,-0.117639
2016-04-06,39.070000,38.340000,39.070000,38.900002,5650000.0,37.276047,1.958361,-0.435112,-0.071301
2016-04-07,39.049999,37.910000,38.660000,38.360001,8206700.0,36.758587,3.101315,-0.775994,-0.279122
2016-04-08,38.720001,38.119999,38.700001,38.500000,6087400.0,36.892742,1.626342,-0.516798,-0.121102
